# Mochan's 3.x Work

- 3.1 Wallets holding OHM (Self Custody)
- 3.2 Wallets holding OHM greater than two months
- 3.3 OHM holder diversity: GINI coefficient

#### Setup Below

In [64]:
from oif.helpers import web3, get_token_transfers, timestamp_of_block
from typing import TypedDict
import pandas as pd
import matplotlib


class Token(TypedDict):
    contract: str
    start_block: int
    decimals: int

ohm = Token(
    contract="0x64aa3364F17a4D01c6f1751Fd97C2BD3D7e7f1D5",
    start_block=13782589,
    decimals=9,
)

sohm = Token(
    contract="0x04906695D6D12CF5459975d7C3C03356E4Ccd460",
    start_block=12622596,
    decimals=9,
)

gohm = Token(
    contract="0x0ab87046fBb341D058F17CBC4c1133F25a20a52f",
    start_block=13674957,
    decimals=18,
)

latest = 14625663

### 3.1 Wallets holding OHM (Self Custody)
Data:
- web3.py (Alchemy):
  - Historical transfers

Calculation:
- Reconstruct wallet balances in-memory using all historical transfers of OHM
  - Include a special attribute to each user's balance called `time_since_zero` (see 3.2)
- Count number of non-zero wallet balances 

In [15]:
ohm_transactions = get_token_transfers(ohm["contract"], startblock=ohm["start_block"], endblock=latest)

get_token_transfers: 13784589-13786589: 0 transfers detected!
get_token_transfers: 13786589-13788589: 0 transfers detected!
get_token_transfers: 13788589-13790589: 0 transfers detected!
get_token_transfers: 13790589-13792589: 0 transfers detected!
get_token_transfers: 13792589-13794589: 0 transfers detected!
get_token_transfers: 13794589-13796589: 0 transfers detected!
get_token_transfers: 13796589-13798589: 0 transfers detected!
get_token_transfers: 13798589-13800589: 0 transfers detected!
get_token_transfers: 13800589-13802589: 0 transfers detected!
get_token_transfers: 13802589-13804589: 0 transfers detected!
get_token_transfers: 13804589-13806589: 0 transfers detected!
get_token_transfers: 13806589-13808589: 122 transfers detected!
get_token_transfers: 13808589-13810589: 273 transfers detected!
get_token_transfers: 13810589-13812589: 359 transfers detected!
get_token_transfers: 13812589-13814589: 526 transfers detected!
get_token_transfers: 13814589-13816589: 613 transfers detected

In [67]:
sohm_transactions = get_token_transfers(sohm["contract"], startblock=sohm["start_block"], endblock=latest)

get_token_transfers: 12624596-12626596: 0 transfers detected!
get_token_transfers: 12626596-12628596: 0 transfers detected!
get_token_transfers: 12628596-12630596: 0 transfers detected!
get_token_transfers: 12630596-12632596: 0 transfers detected!
get_token_transfers: 12632596-12634596: 0 transfers detected!
get_token_transfers: 12634596-12636596: 3 transfers detected!
get_token_transfers: 12636596-12638596: 1635 transfers detected!
get_token_transfers: 12638596-12640596: 1125 transfers detected!
get_token_transfers: 12640596-12642596: 1073 transfers detected!
get_token_transfers: 12642596-12644596: 783 transfers detected!
get_token_transfers: 12644596-12646596: 616 transfers detected!
get_token_transfers: 12646596-12648596: 449 transfers detected!
get_token_transfers: 12648596-12650596: 464 transfers detected!
get_token_transfers: 12650596-12652596: 355 transfers detected!
get_token_transfers: 12652596-12654596: 318 transfers detected!
get_token_transfers: 12654596-12656596: 373 trans

In [52]:
gohm_transactions = get_token_transfers(gohm["contract"], startblock=gohm["start_block"], endblock=latest)

get_token_transfers: 13676957-13678957: 4 transfers detected!
get_token_transfers: 13678957-13680957: 0 transfers detected!
get_token_transfers: 13680957-13682957: 0 transfers detected!
get_token_transfers: 13682957-13684957: 0 transfers detected!
get_token_transfers: 13684957-13686957: 0 transfers detected!
get_token_transfers: 13686957-13688957: 0 transfers detected!
get_token_transfers: 13688957-13690957: 0 transfers detected!
get_token_transfers: 13690957-13692957: 0 transfers detected!
get_token_transfers: 13692957-13694957: 0 transfers detected!
get_token_transfers: 13694957-13696957: 1 transfers detected!
get_token_transfers: 13696957-13698957: 0 transfers detected!
get_token_transfers: 13698957-13700957: 7 transfers detected!
get_token_transfers: 13700957-13702957: 5 transfers detected!
get_token_transfers: 13702957-13704957: 18 transfers detected!
get_token_transfers: 13704957-13706957: 32 transfers detected!
get_token_transfers: 13706957-13708957: 51 transfers detected!
get_t

In [71]:
ohm_df = pd.read_csv("data/ohm_transactions.csv", index_col="block_number").drop("Unnamed: 0", axis=1)
sohm_df = pd.read_csv("data/sohm_transactions.csv", index_col="block_number").drop("Unnamed: 0", axis=1)
gohm_df = pd.read_csv("data/gohm_transactions.csv", index_col="block_number").drop("Unnamed: 0", axis=1)

In [137]:
from collections import defaultdict

def calc_balances(token: Token, df):
    dic = defaultdict(int)
    for _, row in df.iterrows():
        dic[row["from"]] -= int(row["amount"])
        dic[row["to"]] += int(row["amount"])

    return pd.DataFrame([{"address": k, "balance": v / (10**token["decimals"])} for k, v in dic.items()])

In [138]:
ohmies = calc_balances(ohm, ohm_df)
sohmies = calc_balances(sohm, sohm_df)
gohmies = calc_balances(gohm, gohm_df)

In [139]:
ohmies[ohmies["balance"] > 0]["balance"].mean()

5284.464709885201

In [140]:
gohmies

,address,balance
0,0x0ab87046fbb341d058f17cbc4c1133f25a20a52f,-2.312532e+05
1,0xf9704b03e94b8c19cfd8a8803d81c95e814d2a44,0.000000e+00
2,0x0000000000000000000000000000000000000000,1.359877e+05
3,0x5d8b27b839d86edf67a334139e8d51a5348eeba2,1.000000e-09
4,0x0af91fa049a7e1894f480bfe5bba20142c6c29a9,0.000000e+00
...,...,...
72156,0xb7363a2455a63fc99d16ab6ee0808b014e960ab0,1.422881e+02
72157,0x18c082ecc68ed8407bf2b32816d245db68500818,0.000000e+00
72158,0x4df56f8c1d9f6a115ccdf7f084006e3c73944d56,1.818832e-02
72159,0xea7cf159c07b7c8e3a49e53bba8b7a4d5ea7f43e,0.000000e+00


In [145]:
len(ohmies[ohmies["balance"] > 0]) + len(sohmies[sohmies["balance"] > 0]) + len(gohmies[gohmies["balance"] > 0])

75530

In [153]:
d = pd.DataFrame(ohm_transactions)

In [152]:
for row in 

datetime.datetime(2022, 2, 20, 3, 1, 55)